In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pathlib import Path
import xarray as xr
import tensorflow as tf
from shared.models import *
from shared.utilities import *
from shared.training import train_and_evaluate, split_data_on_participants, k_fold_cross_validate
from shared.normalization import *
from shared.generators import *
%env TF_FORCE_GPU_ALLOW_GROWTH=true
%env TF_GPU_ALLOCATOR=cuda_malloc_async

2023-10-13 15:25:24.009346: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-13 15:25:24.995657: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


env: TF_FORCE_GPU_ALLOW_GROWTH=true
env: TF_GPU_ALLOCATOR=cuda_malloc_async


### Set up data

In [2]:
data_path = Path("data/sat1/split_stage_data.nc")

data = xr.load_dataset(data_path)

In [2]:
data_path = Path("data/sat1/split_stage_data_unprocessed_500hz.nc")

data = xr.load_dataset(data_path)

In [3]:
train_data, val_data, test_data = split_data_on_participants(data, 60, norm_dummy)

In [6]:
data

<xarray.Dataset>
Dimensions:      (channels: 30, samples: 770, epochs: 200, participant: 25,
                  labels: 5)
Coordinates:
  * channels     (channels) object 'Fp1' 'Fp2' 'AFz' 'F7' ... 'CPz' 'CP2' 'CP6'
  * samples      (samples) int64 0 1 2 3 4 5 6 7 ... 763 764 765 766 767 768 769
  * epochs       (epochs) int64 0 1 2 3 4 5 6 7 ... 193 194 195 196 197 198 199
  * participant  (participant) object '0001' '0002' '0003' ... '0024' '0025'
  * labels       (labels) object 'confirmation' 'decision' ... 'response'
    stim         (participant, epochs) float64 nan 1.0 1.0 1.0 ... 2.0 nan 2.0
    resp         (participant, epochs) object '' 'resp_left' ... '' 'resp_left'
    RT           (participant, epochs) float64 nan 0.683 1.068 ... nan 1.02
    cue          (participant, epochs) object '' 'SP' 'AC' 'SP' ... 'SP' '' 'AC'
    movement     (participant, epochs) object '' 'stim_left' ... '' 'stim_right'
    trigger      (participant, epochs) object '' ... 'AC/stim_right/resp_left'
Data variables:
    data         (participant, epochs, labels, channels, samples) float64 nan...

In [4]:
model = SAT1Deep(len(data.channels), len(data.samples), len(data.labels))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

2023-10-13 15:25:41.789803: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-13 15:25:41.897261: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-13 15:25:41.897337: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-13 15:25:41.899805: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-13 15:25:41.899870: I tensorflow/compile

In [20]:
model = SAT1TopologicalConv(5, 8, len(data.samples), len(data.labels))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

In [4]:
model = SAT1Base(len(data.channels), len(data.samples), len(data.labels))
model.compile(
    optimizer=tf.keras.optimizers.Nadam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

2023-10-13 11:44:55.222749: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-13 11:44:55.337037: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-13 11:44:55.337108: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-13 11:44:55.339777: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-13 11:44:55.339843: I tensorflow/compile

In [5]:
train_and_evaluate(
    model,
    train_data,
    val_data,
    test_data,
    epochs=20,
    logs_path=Path("logs/")
)

Epoch 1/20


2023-10-13 15:25:51.964293: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-10-13 15:25:53.597572: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-10-13 15:25:55.454579: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-13 15:25:55.464614: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x563b9682dfe0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-13 15:25:55.464641: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-10-13 15:25:55.487818: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:

758/758 [==============================] - 49s 55ms/step - loss: 1.0594 - accuracy: 0.5838 - val_loss: 0.7968 - val_accuracy: 0.7172
Epoch 2/20
758/758 [==============================] - 42s 55ms/step - loss: 0.6904 - accuracy: 0.7455 - val_loss: 0.6313 - val_accuracy: 0.7838
Epoch 3/20
758/758 [==============================] - 42s 55ms/step - loss: 0.5589 - accuracy: 0.7937 - val_loss: 0.5527 - val_accuracy: 0.8069
Epoch 4/20
758/758 [==============================] - 42s 55ms/step - loss: 0.5119 - accuracy: 0.8128 - val_loss: 0.4964 - val_accuracy: 0.8239
Epoch 5/20
758/758 [==============================] - 42s 55ms/step - loss: 0.4728 - accuracy: 0.8249 - val_loss: 0.4737 - val_accuracy: 0.8257
Epoch 6/20
758/758 [==============================] - 42s 55ms/step - loss: 0.4532 - accuracy: 0.8348 - val_loss: 0.4847 - val_accuracy: 0.8255
Epoch 7/20
758/758 [==============================] - 42s 55ms/step - loss: 0.4238 - accuracy: 0.8460 - val_loss: 0.4714 - val_accuracy: 0.8300
Epo

(<keras.src.callbacks.History at 0x7fd4cdc5d890>,
 {'confirmation': {'precision': 0.8676122931442081,
   'recall': 0.835990888382688,
   'f1-score': 0.8515081206496521,
   'support': 439},
  'decision': {'precision': 0.9018475750577367,
   'recall': 0.8905359179019384,
   'f1-score': 0.8961560527825588,
   'support': 877},
  'encoding': {'precision': 0.8027571580063627,
   'recall': 0.8612059158134243,
   'f1-score': 0.8309549945115259,
   'support': 879},
  'pre-attentive': {'precision': 0.7779204107830552,
   'recall': 0.7137809187279152,
   'f1-score': 0.7444717444717445,
   'support': 849},
  'response': {'precision': 0.8514851485148515,
   'recall': 0.8835616438356164,
   'f1-score': 0.8672268907563024,
   'support': 876},
  'accuracy': 0.8380102040816326,
  'macro avg': {'precision': 0.8403245171012429,
   'recall': 0.8370150569323164,
   'f1-score': 0.8380635606343567,
   'support': 3920},
  'weighted avg': {'precision': 0.8376992552952447,
   'recall': 0.8380102040816326,
   'f

In [21]:
train_and_evaluate(
    model,
    train_data,
    val_data,
    test_data,
    epochs=20,
    logs_path=Path("logs/"),
    gen_kwargs={"shape_topological": True},
)

Epoch 1/20
765/765 [==============================] - 12s 14ms/step - loss: 0.7901 - accuracy: 0.6996 - val_loss: 0.6132 - val_accuracy: 0.7712
Epoch 2/20
765/765 [==============================] - 10s 13ms/step - loss: 0.5591 - accuracy: 0.7946 - val_loss: 0.5193 - val_accuracy: 0.8043
Epoch 3/20
765/765 [==============================] - 10s 13ms/step - loss: 0.4688 - accuracy: 0.8308 - val_loss: 0.5023 - val_accuracy: 0.8147
Epoch 4/20
765/765 [==============================] - 10s 13ms/step - loss: 0.4113 - accuracy: 0.8460 - val_loss: 0.4798 - val_accuracy: 0.8274
Epoch 5/20
765/765 [==============================] - 10s 13ms/step - loss: 0.3623 - accuracy: 0.8669 - val_loss: 0.4723 - val_accuracy: 0.8294
Epoch 6/20
765/765 [==============================] - 11s 14ms/step - loss: 0.3210 - accuracy: 0.8834 - val_loss: 0.4800 - val_accuracy: 0.8329
Epoch 7/20
765/765 [==============================] - 11s 14ms/step - loss: 0.2844 - accuracy: 0.8963 - val_loss: 0.4799 - val_accuracy:

(<keras.src.callbacks.History at 0x7fa497865d50>, None)

: 

In [6]:
model.save("models/cnn_base_bigger_conv")

INFO:tensorflow:Assets written to: models/cnn_base_bigger_conv/assets


INFO:tensorflow:Assets written to: models/cnn_base_bigger_conv/assets


In [19]:
# Run before re-training to clear up VRAM
import gc

gc.collect()
tf.keras.backend.clear_session()
del model

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 154, 30, 1)]      0         
                                                                 
 masking (Masking)           (None, 154, 30, 1)        0         
                                                                 
 conv2d (Conv2D)             (None, 150, 30, 64)       384       
                                                                 
 max_pooling2d (MaxPooling2  (None, 75, 30, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 73, 30, 128)       24704     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 30, 128)       0         
 g2D)                                                        